This project's purpose is to build a machine learning model predicting sentiment of a tweet ragarding COVID-19 pandemic, using both "classical" machine learning (like logistic regression ect.) and deep learning methods.

The dataset used in this notebook comes from here: https://www.kaggle.com/datatattle/covid-19-nlp-text-classification
<br>It was collected and manually tagged by a Kaggle user named Aman Miglani.  

In [39]:
import numpy as np
import pandas as pd
import re
from string import punctuation
import nltk
from nltk.corpus import stopwords, words
from nltk.tag import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

In [2]:
STOPWORDS = set(stopwords.words('english'))
ENGLISH_WORDS = set(words.words())
df_train = pd.read_csv(r"data\Corona_NLP_train.csv", encoding='latin1')
df_test = pd.read_csv(r"data\Corona_NLP_test.csv", encoding='latin1')

In [3]:
df_train.head(10)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative
5,3804,48756,"ÃT: 36.319708,-82.363649",16-03-2020,As news of the regionÂs first confirmed COVID...,Positive
6,3805,48757,"35.926541,-78.753267",16-03-2020,Cashier at grocery store was sharing his insig...,Positive
7,3806,48758,Austria,16-03-2020,Was at the supermarket today. Didn't buy toile...,Neutral
8,3807,48759,"Atlanta, GA USA",16-03-2020,Due to COVID-19 our retail store and classroom...,Positive
9,3808,48760,"BHAVNAGAR,GUJRAT",16-03-2020,"For corona prevention,we should stop to buy th...",Negative


In [4]:
print("Size of the train dataset: {}".format(df_train.shape))
print("Size of the test dataset: {}".format(df_test.shape))

Size of the train dataset: (41157, 6)
Size of the test dataset: (3798, 6)


Usually three unique sentiment values are just enough, so I will recode the target variable to such shape.

In [5]:
def recode_sentiment(y):

    if y in ['Extremely Positive', 'Positive']:
        return 'Positive'
    elif y in ['Extremely Negative', 'Negative']:
        return 'Negative'
    else:
        return 'Neutral'

In [6]:
df_train['Sentiment'] = df_train['Sentiment'].apply(lambda x: recode_sentiment(x))
df_test['Sentiment'] = df_test['Sentiment'].apply(lambda x: recode_sentiment(x))

In [7]:
df_train.head(10)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Negative
5,3804,48756,"ÃT: 36.319708,-82.363649",16-03-2020,As news of the regionÂs first confirmed COVID...,Positive
6,3805,48757,"35.926541,-78.753267",16-03-2020,Cashier at grocery store was sharing his insig...,Positive
7,3806,48758,Austria,16-03-2020,Was at the supermarket today. Didn't buy toile...,Neutral
8,3807,48759,"Atlanta, GA USA",16-03-2020,Due to COVID-19 our retail store and classroom...,Positive
9,3808,48760,"BHAVNAGAR,GUJRAT",16-03-2020,"For corona prevention,we should stop to buy th...",Negative


Next cells include cleaning tweets: removing hashtags, URLs, HTML marks, Twitter mentions, stop words and lemmatizing words. </br>

Stop words are frequently occuring words which do not bring much information for our algorithms.</br>

Lemmatization is a process of transforming a word into its root form, for example: running -> run.

In [8]:
def remove_url(string):
    return re.sub(r'https?://\S+|www\.\S+', '', string)

def remove_html(string):
    return re.sub(r'<.*?>', '', string)

def remove_numbers(string):
    return re.sub(r'\d+', '', string)

def remove_mentions(string):
    return re.sub(r'@\w+', '', string)

def remove_hashtags(string):
    return re.sub(r'#\w+', '', string)

def clean_data(tweet, return_tokenized=True):
    
    # Tokenization
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(tweet)
    
    cleaned_tweet = []
    
    for token, tag in pos_tag(tokens):
        
        # Cleaning tokens with regular expressions
        token = remove_url(token)
        token = remove_html(token)
        token = remove_numbers(token)
        token = remove_mentions(token)
        token = remove_hashtags(token)
        
        # Lemmatizing tokens with part of speech recognition
        
        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        
        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)
        
        token = token.lower()
        
        if token not in punctuation and token not in STOPWORDS and token in ENGLISH_WORDS:
            cleaned_tweet.append(token)
    #TfidfVectorizer accepts strings instead of lists of tokens
    if not return_tokenized:
        cleaned_tweet = ' '.join([token for token in cleaned_tweet])

    return cleaned_tweet

In [11]:
df_train['OriginalTweet'] = df_train['OriginalTweet'].apply(lambda x: clean_data(x, return_tokenized=False))
df_test['OriginalTweet'] = df_test['OriginalTweet'].apply(lambda x: clean_data(x, return_tokenized=False))

In [12]:
df_train.head(10)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,,Neutral
1,3800,48752,UK,16-03-2020,advice talk family exchange phone number creat...,Positive
2,3801,48753,Vagabonds,16-03-2020,give elderly disable dedicate shopping hour am...,Positive
3,3802,48754,NaN,16-03-2020,food stock one empty please panic enough food ...,Positive
4,3803,48755,NaN,16-03-2020,ready go supermarket outbreak paranoid food st...,Negative
5,3804,48756,"ÃT: 36.319708,-82.363649",16-03-2020,news first confirm covid case come county last...,Positive
6,3805,48757,"35.926541,-78.753267",16-03-2020,cashier grocery store share insight prove cred...,Positive
7,3806,48758,Austria,16-03-2020,supermarket today buy toilet paper,Neutral
8,3807,48759,"Atlanta, GA USA",16-03-2020,due covid retail store classroom open business...,Positive
9,3808,48760,"BHAVNAGAR,GUJRAT",16-03-2020,corona prevention stop buy thing cash use paym...,Negative


In [13]:
df_train['NumberOfWords'] = df_train['OriginalTweet'].apply(lambda x: len(x.split()))
df_test['NumberOfWords'] = df_test['OriginalTweet'].apply(lambda x: len(x.split()))

In [14]:
df_train.head(10)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment,NumberOfWords
0,3799,48751,London,16-03-2020,,Neutral,0
1,3800,48752,UK,16-03-2020,advice talk family exchange phone number creat...,Positive,22
2,3801,48753,Vagabonds,16-03-2020,give elderly disable dedicate shopping hour am...,Positive,9
3,3802,48754,NaN,16-03-2020,food stock one empty please panic enough food ...,Positive,15
4,3803,48755,NaN,16-03-2020,ready go supermarket outbreak paranoid food st...,Negative,14
5,3804,48756,"ÃT: 36.319708,-82.363649",16-03-2020,news first confirm covid case come county last...,Positive,22
6,3805,48757,"35.926541,-78.753267",16-03-2020,cashier grocery store share insight prove cred...,Positive,12
7,3806,48758,Austria,16-03-2020,supermarket today buy toilet paper,Neutral,5
8,3807,48759,"Atlanta, GA USA",16-03-2020,due covid retail store classroom open business...,Positive,20
9,3808,48760,"BHAVNAGAR,GUJRAT",16-03-2020,corona prevention stop buy thing cash use paym...,Negative,19


In [15]:
df_train = df_train.loc[df_train['NumberOfWords'] > 0,]
df_test = df_test.loc[df_test['NumberOfWords'] > 0,]

In [16]:
df_train.head(10)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment,NumberOfWords
1,3800,48752,UK,16-03-2020,advice talk family exchange phone number creat...,Positive,22
2,3801,48753,Vagabonds,16-03-2020,give elderly disable dedicate shopping hour am...,Positive,9
3,3802,48754,NaN,16-03-2020,food stock one empty please panic enough food ...,Positive,15
4,3803,48755,NaN,16-03-2020,ready go supermarket outbreak paranoid food st...,Negative,14
5,3804,48756,"ÃT: 36.319708,-82.363649",16-03-2020,news first confirm covid case come county last...,Positive,22
6,3805,48757,"35.926541,-78.753267",16-03-2020,cashier grocery store share insight prove cred...,Positive,12
7,3806,48758,Austria,16-03-2020,supermarket today buy toilet paper,Neutral,5
8,3807,48759,"Atlanta, GA USA",16-03-2020,due covid retail store classroom open business...,Positive,20
9,3808,48760,"BHAVNAGAR,GUJRAT",16-03-2020,corona prevention stop buy thing cash use paym...,Negative,19
10,3809,48761,"Makati, Manila",16-03-2020,month crowd supermarket restaurant however red...,Neutral,16


In [17]:
print("Size of the train dataset: {}".format(df_train.shape))
print("Size of the test dataset: {}".format(df_test.shape))

Size of the train dataset: (41052, 7)
Size of the test dataset: (3792, 7)


In [18]:
df_train.drop('NumberOfWords', axis=1, inplace=True)
df_test.drop('NumberOfWords', axis=1, inplace=True)

In [19]:
y_mapping = {'Negative':0, 'Neutral':1, 'Positive':2}
df_train['Sentiment'] = df_train['Sentiment'].map(y_mapping).astype('category')
df_test['Sentiment'] = df_test['Sentiment'].map(y_mapping).astype('category')

In [20]:
df_train.head(10)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
1,3800,48752,UK,16-03-2020,advice talk family exchange phone number creat...,2
2,3801,48753,Vagabonds,16-03-2020,give elderly disable dedicate shopping hour am...,2
3,3802,48754,NaN,16-03-2020,food stock one empty please panic enough food ...,2
4,3803,48755,NaN,16-03-2020,ready go supermarket outbreak paranoid food st...,0
5,3804,48756,"ÃT: 36.319708,-82.363649",16-03-2020,news first confirm covid case come county last...,2
6,3805,48757,"35.926541,-78.753267",16-03-2020,cashier grocery store share insight prove cred...,2
7,3806,48758,Austria,16-03-2020,supermarket today buy toilet paper,1
8,3807,48759,"Atlanta, GA USA",16-03-2020,due covid retail store classroom open business...,2
9,3808,48760,"BHAVNAGAR,GUJRAT",16-03-2020,corona prevention stop buy thing cash use paym...,0
10,3809,48761,"Makati, Manila",16-03-2020,month crowd supermarket restaurant however red...,1


In [21]:
y_train, y_test = df_train['Sentiment'].copy(), df_test['Sentiment'].copy()

X_train_org, X_test_org = df_train['OriginalTweet'].copy(), df_test['OriginalTweet'].copy()

<br>Next cells include training models and hyperparameter tuning.</br>
<br>The tuning involves both TfidfVectorizer and each of the following: logistic regression, multinomial naive Bayes, linear support vector machine.</br>

In [26]:
tdidf_logistic_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__max_features':[100, 200, 300, 400, 600],
               'vect__min_df':[5, 7, 9, 11],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__max_features':[100, 200, 300, 400, 600],
               'vect__min_df':[5, 7, 9, 11],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]

tfidf_logistic_pipeline = Pipeline([
    ('vect', TfidfVectorizer(encoding='latin1', stop_words='english')),
    ('clf', LogisticRegression())
])

cv = StratifiedKFold(n_splits=10)

tfidf_logistic_grid = GridSearchCV(tfidf_logistic_pipeline, param_grid=tdidf_logistic_grid, cv=cv,
                                  verbose=10, n_jobs=-1)

In [27]:
tfidf_logistic_grid.fit(X_train_org, y_train)

Fitting 10 folds for each of 240 candidates, totalling 2400 fits


C:\Users\Mariusz\anaconda3\envs\nlp\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan 0.57173783 0.57173783 0.57173783 0.57178655
 0.62252747 0.62247874 0.62245439 0.62238131 0.64703316 0.64708188
 0.64708188 0.6471306  0.67292704 0.67346297 0.67307321 0.67300012
 0.70547152 0.70573952 0.70537411 0.70554464        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
 0.57146988 0.57146988 0.57146988 0.57144552 0.62206473 0.62208909
 0.62199165 0.62201601 0.64827543 0.6485434  0.6485434  0.64856776
 0.67402322 0.67463221 0.67436427 0.674412

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(encoding='latin1',
                                                        stop_words='english')),
                                       ('clf', LogisticRegression())]),
             n_jobs=-1,
             param_grid=[{'clf__C': [1.0, 10.0, 100.0],
                          'clf__penalty': ['l1', 'l2'],
                          'vect__max_features': [100, 200, 300, 400, 600],
                          'vect__min_df': [5, 7, 9, 11],
                          'vect__ngram_range': [(1, 1)]},
                         {'clf__C': [1.0, 10.0, 100.0],
                          'clf__penalty': ['l1', 'l2'],
                          'vect__max_features': [100, 200, 300, 400, 600],
                          'vect__min_df': [5, 7, 9, 11],
                          'vect__ngram_range': [(1, 1)], 'vect__norm': [Non

In [31]:
print(tfidf_logistic_grid.best_params_)

{'clf__C': 1.0, 'clf__penalty': 'l2', 'vect__max_features': 600, 'vect__min_df': 7, 'vect__ngram_range': (1, 1), 'vect__norm': None, 'vect__use_idf': False}


In [33]:
tfidf_logistic_pipeline = tfidf_logistic_grid.best_estimator_

In [35]:
tfidf_logistic_pipeline.fit(X_train_org, y_train)

C:\Users\Mariusz\anaconda3\envs\nlp\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('vect',
                 TfidfVectorizer(encoding='latin1', max_features=600, min_df=7,
                                 norm=None, stop_words='english',
                                 use_idf=False)),
                ('clf', LogisticRegression())])

In [36]:
def evaluate_model(model, X_train=X_train_org, X_test=X_test_org, y_train=y_train, y_test=y_test):
    
    preds_train = model.predict(X_train)
    preds_test = model.predict(X_test)
    
    train_acc = accuracy_score(y_train, preds_train)
    test_acc = accuracy_score(y_test, preds_test)
    
    return {'Train accuracy':train_acc, 'Test accuracy':test_acc}

In [37]:
print(evaluate_model(model=tfidf_logistic_pipeline))

{'Train accuracy': 0.7292214752021826, 'Test accuracy': 0.696993670886076}


In [38]:
print(classification_report(y_train, tfidf_logistic_pipeline.predict(X_train_org)))
print('-'*80)
print(classification_report(y_test, tfidf_logistic_pipeline.predict(X_test_org)))

              precision    recall  f1-score   support

           0       0.76      0.70      0.73     15392
           1       0.57      0.72      0.64      7620
           2       0.79      0.76      0.77     18040

    accuracy                           0.73     41052
   macro avg       0.71      0.73      0.71     41052
weighted avg       0.74      0.73      0.73     41052

--------------------------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.76      0.67      0.71      1633
           1       0.52      0.67      0.58       613
           2       0.73      0.73      0.73      1546

    accuracy                           0.70      3792
   macro avg       0.67      0.69      0.68      3792
weighted avg       0.71      0.70      0.70      3792



In [42]:
tdidf_naivebayes_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__max_features':[100, 200, 300, 400, 600],
               'vect__min_df':[5, 7, 9, 11],
               'nb__alpha': np.arange(1, 11, 1)},
              {'vect__ngram_range': [(1, 1)],
               'vect__max_features':[100, 200, 300, 400, 600],
               'vect__min_df':[5, 7, 9, 11],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'nb__alpha': np.arange(1, 11, 1)},
              ]
tfidf_naivebayes_pipeline = Pipeline([
    ('vect', TfidfVectorizer(encoding='latin1', stop_words='english')),
    ('nb', MultinomialNB())
])

tfidf_naivebayes_grid = GridSearchCV(tfidf_naivebayes_pipeline, param_grid=tdidf_naivebayes_grid, cv=cv,
                                  verbose=10, n_jobs=-1)

In [43]:
tfidf_naivebayes_grid.fit(X_train_org, y_train)

Fitting 10 folds for each of 400 candidates, totalling 4000 fits


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(encoding='latin1',
                                                        stop_words='english')),
                                       ('nb', MultinomialNB())]),
             n_jobs=-1,
             param_grid=[{'nb__alpha': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
                          'vect__max_features': [100, 200, 300, 400, 600],
                          'vect__min_df': [5, 7, 9, 11],
                          'vect__ngram_range': [(1, 1)]},
                         {'nb__alpha': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
                          'vect__max_features': [100, 200, 300, 400, 600],
                          'vect__min_df': [5, 7, 9, 11],
                          'vect__ngram_range': [(1, 1)], 'vect__norm': [None],
                          'vect__use_idf': [False]

In [44]:
tfidf_naivebayes_pipeline = tfidf_naivebayes_grid.best_estimator_
tfidf_naivebayes_pipeline.fit(X_train_org, y_train)

Pipeline(steps=[('vect',
                 TfidfVectorizer(encoding='latin1', max_features=600, min_df=7,
                                 norm=None, stop_words='english',
                                 use_idf=False)),
                ('nb', MultinomialNB(alpha=1))])

In [45]:
print(evaluate_model(model=tfidf_naivebayes_pipeline))

{'Train accuracy': 0.6422829581993569, 'Test accuracy': 0.6220991561181435}


In [47]:
print(classification_report(y_train, tfidf_naivebayes_pipeline.predict(X_train_org)))
print('-'*80)
print(classification_report(y_test, tfidf_naivebayes_pipeline.predict(X_test_org)))

              precision    recall  f1-score   support

           0       0.67      0.67      0.67     15392
           1       0.46      0.40      0.43      7620
           2       0.68      0.73      0.70     18040

    accuracy                           0.64     41052
   macro avg       0.60      0.60      0.60     41052
weighted avg       0.64      0.64      0.64     41052

--------------------------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.69      0.66      0.67      1633
           1       0.36      0.34      0.35       613
           2       0.65      0.69      0.67      1546

    accuracy                           0.62      3792
   macro avg       0.57      0.56      0.57      3792
weighted avg       0.62      0.62      0.62      3792



In [55]:
tdidf_svm_grid = {
    'svm__penalty': ['l1', 'l2'],
    'svm__C': np.arange(1, 11, 1)
}
tdidf_svm_pipeline = Pipeline([
    ('vect', TfidfVectorizer(encoding='latin1', max_features=600, min_df=7,
                                 norm=None, stop_words='english',
                                 use_idf=False)),
    ('svm', LinearSVC())
])

tdidf_svm_grid = GridSearchCV(tdidf_svm_pipeline, param_grid=tdidf_svm_grid, cv=cv,
                                  verbose=10, n_jobs=-1)

In [56]:
tdidf_svm_grid.fit(X_train_org, y_train)

Fitting 10 folds for each of 20 candidates, totalling 200 fits


C:\Users\Mariusz\anaconda3\envs\nlp\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan 0.71102544        nan 0.71117159        nan 0.7111716
        nan 0.71122031        nan 0.71131774        nan 0.71119595
        nan 0.71131775        nan 0.71175617        nan 0.71080619
        nan 0.71012419]
  warnings.warn(
C:\Users\Mariusz\anaconda3\envs\nlp\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(encoding='latin1',
                                                        max_features=600,
                                                        min_df=7, norm=None,
                                                        stop_words='english',
                                                        use_idf=False)),
                                       ('svm', LinearSVC())]),
             n_jobs=-1,
             param_grid={'svm__C': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
                         'svm__penalty': ['l1', 'l2']},
             verbose=10)

In [57]:
tdidf_svm_pipeline = tdidf_svm_grid.best_estimator_
tdidf_svm_pipeline.fit(X_train_org, y_train)

C:\Users\Mariusz\anaconda3\envs\nlp\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


Pipeline(steps=[('vect',
                 TfidfVectorizer(encoding='latin1', max_features=600, min_df=7,
                                 norm=None, stop_words='english',
                                 use_idf=False)),
                ('svm', LinearSVC(C=8))])

In [59]:
print(evaluate_model(model=tdidf_svm_pipeline))

{'Train accuracy': 0.7285637727759914, 'Test accuracy': 0.6959388185654009}


In [60]:
print(classification_report(y_train, tdidf_svm_pipeline.predict(X_train_org)))
print('-'*80)
print(classification_report(y_test, tdidf_svm_pipeline.predict(X_test_org)))

              precision    recall  f1-score   support

           0       0.76      0.71      0.73     15392
           1       0.58      0.69      0.63      7620
           2       0.79      0.76      0.77     18040

    accuracy                           0.73     41052
   macro avg       0.71      0.72      0.71     41052
weighted avg       0.74      0.73      0.73     41052

--------------------------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.76      0.67      0.71      1633
           1       0.52      0.65      0.58       613
           2       0.72      0.74      0.73      1546

    accuracy                           0.70      3792
   macro avg       0.67      0.69      0.67      3792
weighted avg       0.71      0.70      0.70      3792



<br>So far, logistic regression and linear SVM are performing the best.</br>
<br>The results are not so bad, but there surely is a space for improvement.</br>